<div class="markdown-google-sans">

# <strong>事先準備</strong>
</div>

<div class="markdown-google-sans">

## <strong>安裝plotly</strong>
</div>
範例會使用plotly表格呈現,請先安裝plotly<br/>


In [ ]:
!pip install plotly

<div class="markdown-google-sans">

## <strong>載入plotly和設定渲染器</strong>
</div>
並且為了colab可以瀏覽,請透過以下指令設定

In [ ]:
#匯入顯示表格套件
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
import plotly.io as pio
pio.renderers.default = 'colab'  # 或嘗試 'notebook' 或 'iframe' 渲染器
from google.colab import output
output.enable_custom_widget_manager()
init_notebook_mode(connected=True)



<div class="markdown-google-sans">

## <strong>範例欄位設定</strong>
</div>
定義顯示欄位<br/>

In [4]:
from typing import List
#匯入顯示資料套件
import pandas as pd
from IPython.display import display,clear_output
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
from IPython.display import display, clear_output

# 初始數據
head_match = dict(values=['成交時間', '第一成交價','第一成交量','第2到第N成交價','第2到第N成交量'])
head_bs5 = dict(values=['買價1','買量1' ,'買價2','買量2' ,'買價3','買量3' ,'買價4','買量4' ,'買價5','買量5',
             '賣價1','賣量1' ,'賣價2','賣量2' ,'賣價3','賣量3' ,'賣價4','賣量4' ,'賣價5','賣量5'])
head_open = dict(values=['開盤時間','開盤價' ,'開盤量','最高低價時間' ,'最高','最低'])
head_idx = dict(values=['現貨統計時間','現貨價格'])
head_settle = dict(values=['該期最高價','該期最低價','當日最高價','當日最低價','開盤價','最後買價','最後賣價','收盤價','委託買進總筆數','委託買進總口數','委託賣出總筆數','委託賣出總口數','總成交筆數','總成交口數','合併委託買進總筆數','合併委託買進總口數','合併委託賣出總筆數','合併委託賣出總口數','合併總成交量','結算價','未平倉合約數','鉅額交易總成交量'])

# 創建初始表格
fig_match = go.FigureWidget(data=[go.Table( header=head_match, cells=dict(values=list(zip(*[['','','','','']])), align='left'),)])
fig_match.update_layout(height=100, margin=dict(t=0, b=0, l=0, r=0),autosize=True)
fig_bs5 = go.FigureWidget(data=[go.Table( header=head_bs5, cells=dict(values=list(zip(*[['','','','','','','','','','','','','','','','','','','','']])), align='left'),)])
fig_bs5.update_layout(height=100, margin=dict(t=0, b=0, l=0, r=0),autosize=True)
fig_open = go.FigureWidget(data=[go.Table( header=head_open, cells=dict(values=list(zip(*[['','','','','','']])), align='left'),)])
fig_open.update_layout(height=100, margin=dict(t=0, b=0, l=0, r=0),autosize=True)
fig_idx = go.FigureWidget(data=[go.Table( header=head_idx, cells=dict(values=list(zip(*[['','']] )), align='left'),)])
fig_idx.update_layout(height=100, margin=dict(t=0, b=0, l=0, r=0),autosize=True)
fig_settle = go.FigureWidget(data=[go.Table( header=head_settle, cells=dict(values=list(zip(*[['','','','','','','','','','','','','','','','','','','','','','']] )), align='left'),)])
fig_settle.update_layout(height=100, margin=dict(t=0, b=0, l=0, r=0),autosize=True)

col_match = {header: idx for idx, header in enumerate(head_match['values'])}
col_bs5 = {header: idx for idx, header in enumerate(head_bs5['values'])}
col_open = {header: idx for idx, header in enumerate(head_open['values'])}
col_idx = {header: idx for idx, header in enumerate(head_idx['values'])}
col_settle = {header: idx for idx, header in enumerate(head_settle['values'])}

# 初始K線圖表格
bardata = [['', '', '', '', '', '', '', '', '']]
bardata_dd = pd.DataFrame(bardata)
bardata_dd.columns = ['商品代號', '商品種類', '日期', '時間', '開', '高', '低', '收', '成交量']

<div class="markdown-google-sans">

## <strong>安裝unitrade</strong>
</div>

In [ ]:
!pip install unitrade

<div class="markdown-google-sans">

# <strong>初始化</strong>
</div>

In [ ]:
##API初始&登入
import unitrade
from unitrade.unitrade import *
from unitrade.quote.ddata import *
#初始API
api = Unitrade()

<div class="markdown-google-sans">

# <strong>登入api</strong>
</div>

In [ ]:
#錯誤事件處理
api.on_error = lambda error: print(f'api錯誤 {error}')
#系統登入
loginresponse = api.login(
    "url", "帳號", "密碼", '憑證檔名', '憑證密碼')#url,帳號,密碼,憑證檔名,憑證密碼


#顯示登入結果
print(loginresponse)
print(f"登入旗標{api.login_status_flag}")
print(f"交易帳號{api.get_accounts()}")


<div class="markdown-google-sans">

# <strong>查詢行情</strong>
</div>


<div class="markdown-google-sans">

## <strong>成交價量</strong>
</div>
透過query_tick_data_trade方法實作,帶入商品代碼


In [ ]:
tickDataTradeResponse=api.dquote.query_tick_data_trade('TXFH5')
if tickDataTradeResponse.ok == True:
    tickdatatrade=tickDataTradeResponse.data
    cur = list(fig_match.data[0].cells.values)
    cur[col_match['成交時間']] = tickdatatrade.matchtime
    cur[col_match['第一成交價']] = tickdatatrade.matchprice
    cur[col_match['第一成交量']] = tickdatatrade.matchquantity
    if len(tickdatatrade.matchpricedata)>0:
        cur[col_match['第2到第N成交價']] =",".join(map(str, tickdatatrade.matchpricedata))
        cur[col_match['第2到第N成交量']] = ",".join(map(str, tickdatatrade.matchqtydata)) 
    with fig_match.batch_update():
        fig_match.data[0].cells.values = cur
    display(fig_match)
else:
    print("錯誤訊息 "+tickDataTradeResponse.error)

<div class="markdown-google-sans">

## <strong>最佳買賣價量</strong>
</div>
透過query_tick_data_bid_offer方法實作,帶入商品代碼


In [ ]:
tickDataBeforeBidOfferResponse=api.dquote.query_tick_data_bid_offer('TXFH5')
if tickDataBeforeBidOfferResponse.ok == True:
    tickdatabidoffer=tickDataBeforeBidOfferResponse.data
    cur = list(fig_bs5.data[0].cells.values)
    cur[col_bs5['買價1']] = tickdatabidoffer.bp1
    cur[col_bs5['買量1']] = tickdatabidoffer.bq1
    cur[col_bs5['買價2']] = tickdatabidoffer.bp2
    cur[col_bs5['買量2']] = tickdatabidoffer.bq2
    cur[col_bs5['買價3']] = tickdatabidoffer.bp3
    cur[col_bs5['買量3']] = tickdatabidoffer.bq3
    cur[col_bs5['買價4']] = tickdatabidoffer.bp4
    cur[col_bs5['買量4']] = tickdatabidoffer.bq4
    cur[col_bs5['買價5']] = tickdatabidoffer.bp5
    cur[col_bs5['買量5']] = tickdatabidoffer.bq5
    cur[col_bs5['賣價1']] = tickdatabidoffer.sp1
    cur[col_bs5['賣量1']] = tickdatabidoffer.sq1
    cur[col_bs5['賣價2']] = tickdatabidoffer.sp2
    cur[col_bs5['賣量2']] = tickdatabidoffer.sq2
    cur[col_bs5['賣價3']] = tickdatabidoffer.sp3
    cur[col_bs5['賣量3']] = tickdatabidoffer.sq3
    cur[col_bs5['賣價4']] = tickdatabidoffer.sp4
    cur[col_bs5['賣量4']] = tickdatabidoffer.sq4
    cur[col_bs5['賣價5']] = tickdatabidoffer.sp5
    cur[col_bs5['賣量5']] = tickdatabidoffer.sq5
    with fig_bs5.batch_update():
        fig_bs5.data[0].cells.values = cur
    display(fig_bs5)
else:
    print("tickdatabidoffer "+tickDataBeforeBidOfferResponse.error)


<div class="markdown-google-sans">

## <strong>開盤價價量</strong>
</div>
透過query_tick_data_open方法實作,帶入商品代碼


In [ ]:
tickdataopenResponse=api.dquote.query_tick_data_open('TXFH5')
if tickdataopenResponse.ok == True:
    tickdataopen=tickdataopenResponse.data
    cur = list(fig_open.data[0].cells.values)
    cur[col_open['開盤時間']] = tickdataopen.opentime
    cur[col_open['開盤價']] = tickdataopen.openprice
    cur[col_open['開盤量'] ]= tickdataopen.openquantity
    with fig_open.batch_update():
        fig_open.data[0].cells.values = cur
    display(fig_open)
else:
    print("錯誤訊息 "+tickdataopenResponse.error)


<div class="markdown-google-sans">

## <strong>最高最低價</strong>
</div>
透過query_tick_data_high_low方法實作,帶入商品代碼

In [ ]:
tickDataHignLowResponse=api.dquote.query_tick_data_high_low('TXFH5')
if tickDataHignLowResponse.ok == True:
    tickdatahighlow=tickDataHignLowResponse.data
    cur = list(fig_open.data[0].cells.values)
    cur[col_open['最高']] = tickdatahighlow.dayhighprice
    cur[col_open['最低']] = tickdatahighlow.daylowprice
    cur[col_open['最高低價時間']] = tickdatahighlow.showtime
    with fig_open.batch_update():
        fig_open.data[0].cells.values = cur
    display(fig_open)
else:
    print("錯誤 "+tickDataTradeResponse.error)




<div class="markdown-google-sans">

## <strong>現貨價量</strong>
</div>
透過query_index_data方法實作,帶入標的物代碼


In [ ]:
#查詢現貨價量範例
index_id='TXF'#代碼
indexdataResponse=api.dquote.query_index_data(index_id)
if indexdataResponse.ok == True:
  indexdata=indexdataResponse.data
  cur = list(fig_idx.data[0].cells.values)
  cur[col_idx['現貨統計時間']] = indexdata.index_time
  cur[col_idx['現貨價格']] = indexdata.index_value
  with fig_idx.batch_update():
    fig_idx.data[0].cells.values = cur
    display(fig_idx)
else:
  print(indexdataResponse.error)


<div class="markdown-google-sans">

## <strong>收盤行情資料訊息含結算價及未平倉合約數(在V1.0.0.4版本開始提供)</strong>
</div>
透過query_tick_data_settle方法實作,帶入商品代碼

In [ ]:
#查詢收盤行情資料訊息含結算價及未平倉合約數範例 
settledataResponse=api.dquote.query_tick_data_settle("TXFH5")
if settledataResponse.ok == True:
  settledata=settledataResponse.data
  cur = list(fig_settle.data[0].cells.values)
  cur[col_settle['該期最高價']] = settledata.period_high_price
  cur[col_settle['該期最低價']] = settledata.period_low_price
  cur[col_settle['當日最高價']] = settledata.daily_high_price
  cur[col_settle['當日最低價']] = settledata.daily_low_price
  cur[col_settle['開盤價']] = settledata.open_price
  cur[col_settle['最後買價']] = settledata.last_bid_price
  cur[col_settle['最後賣價']] = settledata.last_ask_price
  cur[col_settle['收盤價']] = settledata.close_price
  cur[col_settle['委託買進總筆數']] = settledata.total_bid_orders
  cur[col_settle['委託買進總口數']] = settledata.total_bid_volume
  cur[col_settle['委託賣出總筆數']] = settledata.total_ask_orders
  cur[col_settle['委託賣出總口數']] = settledata.total_ask_volume
  cur[col_settle['總成交筆數']] = settledata.total_trades
  cur[col_settle['總成交口數']] = settledata.total_trade_volume
  cur[col_settle['合併委託買進總筆數']] = settledata.merged_bid_orders
  cur[col_settle['合併委託買進總口數']] = settledata.merged_bid_volume
  cur[col_settle['合併委託賣出總筆數']] = settledata.merged_ask_orders

  cur[col_settle['合併委託賣出總口數']] = settledata.merged_ask_volume
  cur[col_settle['合併總成交量']] = settledata.merged_volume
  cur[col_settle['結算價']] = settledata.settlement_price
  cur[col_settle['未平倉合約數']] = settledata.open_interest
  cur[col_settle['鉅額交易總成交量']] = settledata.block_trade_volume
  with fig_settle.batch_update():
    fig_settle.data[0].cells.values = cur
    display(fig_settle)
else:
  print(settledataResponse.error)

<div class="markdown-google-sans">

# <strong>接收即時行情</strong>
</div>

<div class="markdown-google-sans">

## <strong>*成交和最佳買賣價*</strong>
</div>

<div class="markdown-google-sans">

### <strong>*註冊*</strong>
</div>

In [ ]:
#成交價量接收處理
def on_tick_data_trade(tickdatatrade: DTickDataTrade):
    cur = list(fig_match.data[0].cells.values)
    cur[col_match['成交時間']] = tickdatatrade.matchtime
    cur[col_match['第一成交價']] = tickdatatrade.matchprice
    cur[col_match['第一成交量']] = tickdatatrade.matchquantity
    
    if len(tickdatatrade.matchpricedata)>0:
        cur[col_match['第2到第N成交價']] =",".join(map(str, tickdatatrade.matchpricedata))
        cur[col_match['第2到第N成交量']] = ",".join(map(str, tickdatatrade.matchqtydata)) 
    with fig_match.batch_update():
        fig_match.data[0].cells.values = cur
#最佳買賣價量接收處理
def on_tick_data_bid_offer(tickdatabidoffer: DTickDataBidOffer):
    cur = list(fig_bs5.data[0].cells.values)
    cur[col_bs5['買價1']] = tickdatabidoffer.bp1
    cur[col_bs5['買量1']] = tickdatabidoffer.bq1
    cur[col_bs5['買價2']] = tickdatabidoffer.bp2
    cur[col_bs5['買量2']] = tickdatabidoffer.bq2
    cur[col_bs5['買價3']] = tickdatabidoffer.bp3
    cur[col_bs5['買量3']] = tickdatabidoffer.bq3
    cur[col_bs5['買價4']] = tickdatabidoffer.bp4
    cur[col_bs5['買量4']] = tickdatabidoffer.bq4
    cur[col_bs5['買價5']] = tickdatabidoffer.bp5
    cur[col_bs5['買量5']] = tickdatabidoffer.bq5
    cur[col_bs5['賣價1']] = tickdatabidoffer.sp1
    cur[col_bs5['賣量1']] = tickdatabidoffer.sq1
    cur[col_bs5['賣價2']] = tickdatabidoffer.sp2
    cur[col_bs5['賣量2']] = tickdatabidoffer.sq2
    cur[col_bs5['賣價3']] = tickdatabidoffer.sp3
    cur[col_bs5['賣量3']] = tickdatabidoffer.sq3
    cur[col_bs5['賣價4']] = tickdatabidoffer.sp4
    cur[col_bs5['賣量4']] = tickdatabidoffer.sq4
    cur[col_bs5['賣價5']] = tickdatabidoffer.sp5
    cur[col_bs5['賣量5']] = tickdatabidoffer.sq5
    with fig_bs5.batch_update():
        fig_bs5.data[0].cells.values = cur
#訂閱接收事件
api.dquote.on_tick_data_trade = on_tick_data_trade
api.dquote.on_tick_data_bid_offer = on_tick_data_bid_offer
subok,msg=api.dquote.subscribe_trade_bid_offer('TXFH5')
print(f'註冊結果 {subok} {msg}')
display(fig_match)
display(fig_bs5)

<div class="markdown-google-sans">

### <strong>反註冊</strong>

</div>

In [ ]:
unsub_ok,msg=api.dquote.unsubscribe_trade_bid_offer('TXFH5')
print(f'反註冊結果  {unsub_ok} {msg}')

<div class="markdown-google-sans">

## <strong>*開盤價量*</strong>
</div>

<div class="markdown-google-sans">

### <strong>*註冊*</strong>
</div>

In [ ]:
productid='TXFH5'#商品代碼
#開盤價量接收處理
def on_tick_data_open(tickdataopen: DTickDataOpen):
    cur = list(fig_open.data[0].cells.values)
    cur[col_open['開盤時間']] = tickdataopen.opentime
    cur[col_open['開盤價']] = tickdataopen.openprice
    cur[col_open['開盤量'] ]= tickdataopen.openquantity
    with fig_open.batch_update():
        fig_open.data[0].cells.values = cur

#訂閱接收事件
api.dquote.on_tick_data_open = on_tick_data_open
subok,msg=api.dquote.subscribe_open(productid)
print(f'註冊結果 {productid} {subok} {msg}')
display(fig_open)

<div class="markdown-google-sans">

### <strong>*反註冊*</strong>
</div>

In [ ]:

unsub_ok,msg=api.dquote.unsubscribe_open(productid)
print(f'反註冊結果 {productid} {unsub_ok} {msg}')


<div class="markdown-google-sans">

## <strong>*最高最低價量*</strong>
</div>

<div class="markdown-google-sans">

### <strong>*註冊*</strong>
</div>

In [ ]:
#最高最低價接收處理
def on_tick_data_high_low(tickdatahighlow: DTickDataHighLow):
    cur = list(fig_open.data[0].cells.values)
    cur[col_open['最高']] = tickdatahighlow.dayhighprice
    cur[col_open['最低']] = tickdatahighlow.daylowprice
    cur[col_open['最高低價時間']] = tickdatahighlow.showtime
    with fig_open.batch_update():
        fig_open.data[0].cells.values = cur


#訂閱接收事件
api.dquote.on_tick_data_high_low = on_tick_data_high_low
subok,msg=api.dquote.subscribe_high_low(productid)
print(f'註冊結果 {productid} {subok} {msg}')
display(fig_open)

<div class="markdown-google-sans">

### <strong>*反註冊*</strong>
</div>

In [ ]:
unsub_ok,msg=api.dquote.unsubscribe_high_low(productid)
print(f'反註冊結果 {productid} {unsub_ok} {msg}')

<div class="markdown-google-sans">

## <strong>*現貨報價*</strong>
</div>


<div class="markdown-google-sans">

### <strong>*註冊*</strong>
</div>

In [ ]:
#訂閱現貨價量接收範例
index_id='TXF' #代碼
#接收現貨價量處理
def on_index_data(indexdata: DIndexData):
    cur = list(fig_idx.data[0].cells.values)
    cur[col_idx['現貨統計時間']] = indexdata.index_time
    cur[col_idx['現貨價格']] = indexdata.index_value
    with fig_idx.batch_update():
        fig_idx.data[0].cells.values = cur
#訂閱事件
api.dquote.on_index_data = on_index_data
#註冊報價
subok,msg=api.dquote.subscribe_index_data('1',index_id)
print(f'註冊結果 {index_id} {subok} {msg}')
display(fig_idx)

<div class="markdown-google-sans">

### <strong>*反註冊*</strong>
</div>

In [ ]:
#反註冊現貨價量接收
unsub_ok,msg=api.dquote.unsubscribe_index_data('1',index_id)
print(f'反註冊結果 {index_id} {unsub_ok} {msg}')

<div class="markdown-google-sans">

## <strong>*收盤行情資料訊息含結算價及未平倉合約數(在V1.0.0.4版本開始提供)*</strong>
</div>

<div class="markdown-google-sans">

### <strong>*註冊*</strong>
</div>

In [ ]:
#收盤行情資料訊息含結算價及未平倉合約數處理
def on_tick_data_settle(tickdatasettle: DTickDataSettle):
    cur = list(fig_settle.data[0].cells.values)  
    cur[col_settle['該期最高價']] = settledata.period_high_price
    cur[col_settle['該期最低價']] = settledata.period_low_price
    cur[col_settle['當日最高價']] = settledata.daily_high_price
    cur[col_settle['當日最低價']] = settledata.daily_low_price
    cur[col_settle['開盤價']] = settledata.open_price
    cur[col_settle['最後買價']] = settledata.last_bid_price
    cur[col_settle['最後賣價']] = settledata.last_ask_price
    cur[col_settle['收盤價']] = settledata.close_price
    cur[col_settle['委託買進總筆數']] = settledata.total_bid_orders
    cur[col_settle['委託買進總口數']] = settledata.total_bid_volume
    cur[col_settle['委託賣出總筆數']] = settledata.total_ask_orders
    cur[col_settle['委託賣出總口數']] = settledata.total_ask_volume
    cur[col_settle['總成交筆數']] = settledata.total_trades
    cur[col_settle['總成交口數']] = settledata.total_trade_volume
    cur[col_settle['合併委託買進總筆數']] = settledata.merged_bid_orders
    cur[col_settle['合併委託買進總口數']] = settledata.merged_bid_volume
    cur[col_settle['合併委託賣出總筆數']] = settledata.merged_ask_orders

    cur[col_settle['合併委託賣出總口數']] = settledata.merged_ask_volume
    cur[col_settle['合併總成交量']] = settledata.merged_volume
    cur[col_settle['結算價']] = settledata.settlement_price
    cur[col_settle['未平倉合約數']] = settledata.open_interest
    cur[col_settle['鉅額交易總成交量']] = settledata.block_trade_volume
    with fig_settle.batch_update():
        fig_settle.data[0].cells.values = cur
    cur = list(fig_settle.data[0].cells.values)
#訂閱接收事件
productid='TXFH5'
api.dquote.on_tick_data_settle = on_tick_data_settle
subok,msg=api.dquote.subscribe_settle(productid)
print(f'註冊結果 {productid} {subok} {msg}')
display(fig_settle)

<div class="markdown-google-sans">

### <strong>*反註冊*</strong>
</div>

In [ ]:
unsub_ok,msg=api.dquote.unsubscribe_open(productid)
print(f'反註冊結果 {productid} {unsub_ok} {msg}')


<div class="markdown-google-sans">

## <strong>*查詢歷史K線行情(在V1.0.0.4版本開始提供)*</strong>
</div>

<div class="markdown-google-sans">

### <strong>*日K*</strong>
</div>

In [ ]:
interval = "D"  # K線週期 D:日K、1K:分K
start = datetime(2025, 4, 1, 0, 0, 0)
end = datetime(2025, 6, 27, 0, 0, 0)
productkind = "1"  # 商品種類 1:期貨 2:選擇權 4:期貨價差
productId = "MXFH5"  # 商品代號
count = 50 # 取得K線數量
bardatas = api.dquote.get_history_bardata(interval, start, end, productkind, productId, count)
if (bardatas.ok):
    i = 0
    for bardata in bardatas.data:
        bardata_dd.loc[i] = [bardata.productId, bardata.productkind, bardata.date, bardata.time, bardata.open, bardata.high, bardata.low, bardata.close, bardata.volume]
        i += 1
    clear_output(wait=True)
    display(bardata_dd)
else:
    print("查詢發生錯誤: " + bardatas.error)

<div class="markdown-google-sans">

### <strong>*分K*</strong>
</div>

In [ ]:
interval = "1K"  # K線週期 D:日K、1K:分K
start = datetime(2025, 6, 27, 8, 45, 0)
end = datetime(2025, 6, 27, 10, 15, 0)
productkind = "1"  # 商品種類 1:期貨 2:選擇權 4:期貨價差
productId = "TXFH5"  # 商品代號
count = 50 # 取得K線數量
bardatas = api.dquote.get_history_bardata(interval, start, end, productkind, productId, count)
if (bardatas.ok):
    i = 0
    for bardata in bardatas.data:
        bardata_dd.loc[i] = [bardata.productId, bardata.productkind, bardata.date, bardata.time, bardata.open, bardata.high, bardata.low, bardata.close, bardata.volume]
        i += 1
    clear_output(wait=True)
    display(bardata_dd)
else:
    print("查詢發生錯誤: " + bardatas.error)

<div class="markdown-google-sans">

# <strong>*系統登出*</strong>
</div>


In [ ]:
#系統登出
api.logout()
print('登出完畢')
print(f"登入旗標{api.login_status_flag}")
